In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import open_clip

from upyog.all import *
if True:
    import sys
    sys.path.append("/home/synopsis/git/CinemaNet-Training/")
    sys.path.append("/home/synopsis/git/YOLOX-Custom/")
    sys.path.append("/home/synopsis/git/YOLO-CinemaNet/")
    sys.path.append("/home/synopsis/git/icevision/")
    sys.path.append("/home/synopsis/git/labelling-workflows/")
    sys.path.append("/home/synopsis/git/amalgam/")
    sys.path.append("/home/synopsis/git/cinemanet-multitask-classification/")
    sys.path.append("/home/synopsis/git/Synopsis.py/")

In [ ]:
from cinemanet.CLIP.inference import *
from cinemanet.CLIP.utils import *

In [ ]:
torch.cuda.set_device(0)

#### Load Models

In [ ]:
variant = 'ViT-L-14'
device = 2

ckpt_path = "./logs/2022_12_14-08_05_35-model_ViT-L-14-lr_0.0001-b_56-j_8-p_amp_bf16/checkpoints/epoch_3.pt"
alpha = 0.5

In [ ]:
tokenizer = open_clip.get_tokenizer(variant)

model_stock = load_model(variant, "cpu", 'openai')
model_finetune = load_model(variant, "cpu", None, ckpt_path)

sd_stock = model_stock.state_dict()
sd_finetune = model_finetune.state_dict()
blended_wts = interpolate_weights(sd_stock, sd_finetune, alpha)

model_stock = load_model(variant, device, 'openai')
model_finetune = load_model(variant, device, None, ckpt_path)

model_finetune.load_state_dict(blended_wts)

#### Run Inference

In [ ]:
path_stock_embedding = "/home/synopsis/datasets/serialised-datasets/CLIP/ViT-L14__shotdeck-embeddings.feather"
path_finetune_embedding = "/home/synopsis/datasets/serialised-datasets/CLIP/ViT-L14__finetune-last-5__1-epoch__multicap__alpha-0.5__shotdeck-embeddings.feather"

df_stock = pd.read_feather(path_stock_embedding)
df_finetune = pd.read_feather(path_finetune_embedding)

In [ ]:
filt = df_stock.filepath.isin(df_finetune.filepath)
df_stock = df_stock[filt]

filt = df_finetune.filepath.isin(df_stock.filepath)
df_finetune = df_finetune[filt]

df_stock = df_stock.sort_values("filepath").reset_index(drop=True)
df_finetune = df_finetune.sort_values("filepath").reset_index(drop=True)

#### View Top Matches

In [ ]:
stock_embeddings = np.stack(df_stock.embedding)
finetune_embeddings = np.stack(df_finetune.embedding)

stock_embeddings.shape, finetune_embeddings.shape

In [ ]:
rich.print(EVALUATION_PROMPTS)

In [ ]:
N = 21

for prompt in tqdm(EVALUATION_PROMPTS, "Visualising Results..."):
    top_stock = get_top_matches(prompt, model_stock, stock_embeddings, tokenizer)
    top_finetune = get_top_matches(prompt, model_finetune, finetune_embeddings, tokenizer)

    matches_stock = view_top_matches(df_stock, top_stock, N)
    matches_finetune = view_top_matches(df_finetune, top_finetune, N)

    sep = Image.new("RGB", (50, matches_stock.height), color=(255,255,255))

    rich.print(f"QUERY: '{query}'")
    display(matches_stock | sep | matches_finetune)
    
    rich.print("-" * 90)